In [ ]:
!pip install datasets transformers tokenizers torch scikit-learn -q

# LAB 2: SI700NLP

**Elaborado por**: Andres Felipe Restrepo Acevedo

In [ ]:
pip install accelerate -q # pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from transformers import AutoTokenizer, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

/Users/andrestrepo/Documents/repos_personal/si7003nlp_eafit/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, processors

In [4]:
from transformers import PreTrainedTokenizerFast

In [5]:
# Load AG_NEWS dataset using the datasets library
dataset = load_dataset("ag_news")
dataset_train = dataset["train"]
dataset_test = dataset["test"]

In [6]:
len(dataset_train)

120000

In [7]:
# Initialize a BPE tokenizer
tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))

# Set the pre-tokenizer to split on whitespace
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Set up the trainer with special tokens
trainer = trainers.BpeTrainer(
    vocab_size=1000,
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

# Define a generator to yield batches of text
def batch_iterator(batch_size=1000):
    for i in range(0, len(dataset_train), batch_size):
        yield dataset_train[i:i + batch_size]["text"]

# Train the tokenizer
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

# Set the post-processor to add special tokens
tokenizer.post_processor = processors.TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", tokenizer.token_to_id("[CLS]")),
        ("[SEP]", tokenizer.token_to_id("[SEP]")),
    ],
)
# Enable truncation and padding
tokenizer.enable_truncation(max_length=128)
tokenizer.enable_padding(pad_id=tokenizer.token_to_id("[PAD]"), pad_token="[PAD]", length=128)

In [8]:
tokenizer.save("custom_tokenizer.json")

In [9]:
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file="custom_tokenizer.json")

In [10]:
def preprocess_data(examples):
    # Tokenize the text
    encodings = tokenizer.encode_batch(examples["text"])

    # Extract input IDs and attention masks
    input_ids = [encoding.ids for encoding in encodings]
    attention_masks = [encoding.attention_mask for encoding in encodings]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_masks,
        "label": examples["label"]
    }


In [11]:
# Apply preprocessing
ds_train = dataset_train.map(preprocess_data, batched=True)
ds_test = dataset_test.map(preprocess_data, batched=True)

In [12]:
ds_train[0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2,
 'input_ids': [1,
  54,
  150,
  202,
  16,
  33,
  996,
  34,
  699,
  33,
  215,
  157,
  99,
  95,
  923,
  215,
  11,
  219,
  12,
  219,
  15,
  383,
  692,
  15,
  78,
  245,
  133,
  14,
  54,
  150,
  202,
  104,
  124,
  10,
  78,
  63,
  82,
  286,
  71,
  101,
  58,
  61,
  112,
  103,
  745,
  144,
  15,
  62,
  84,
  73,
  605,
  14,
  192,
  151,
  64,
  101,
  66,
  104,
  96,
  362,
  16,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  

In [35]:
# Define Basic Embedding Model
class BasicEmbeddingModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=3)
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, input_ids, attention_mask, labels=None):
        embedded = self.embedding(input_ids)
        embedded = torch.mean(embedded, dim=1)  # Averaging embeddings
        logits = self.fc(embedded)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}

In [36]:
# Model setup
embed_dim = 100
num_classes = 4
vocab_size = 1000
model = BasicEmbeddingModel(vocab_size, embed_dim, num_classes)
# Show model summary
print(model)

BasicEmbeddingModel(
  (embedding): Embedding(1000, 100, padding_idx=3)
  (fc): Linear(in_features=100, out_features=4, bias=True)
)


In [37]:
# Define compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1}

In [41]:
# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-2,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
)

In [42]:
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_test,
    processing_class=fast_tokenizer,
    compute_metrics=compute_metrics
)

In [43]:
# Train model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.421100,0.428026,0.848421,0.847990
2,0.406400,0.423046,0.849079,0.848574
3,0.402900,0.416045,0.851316,0.850995
4,0.388500,0.414268,0.854211,0.854063
5,0.381600,0.413700,0.852500,0.852185


TrainOutput(global_step=18750, training_loss=0.3984001985677083, metrics={'train_runtime': 124.6024, 'train_samples_per_second': 4815.315, 'train_steps_per_second': 150.479, 'total_flos': 0.0, 'train_loss': 0.3984001985677083, 'epoch': 5.0})

# Lab: Enhancing a Basic Embedding Model with Positional Encoding and Multi-Head Attention

## Objective
In this lab, you will modify a basic sentiment analysis model by adding two key components from transformer-based architectures: **positional encoding** and **multi-head attention**. These modifications will improve the model’s ability to capture word order and token interactions, making it more expressive.

---

## Task Overview
You are provided with a basic embedding model that performs sentiment analysis by:
- Mapping token IDs to embeddings.
- Averaging embeddings across tokens.
- Passing the result through a fully connected layer for classification.

Your tasks:
1. **Implement positional encoding** to enrich word embeddings with information about token positions.
2. **Replace the mean pooling operation** with a **multi-head self-attention layer** to allow tokens to attend to each other.

---

## Step 1: Implement Positional Encoding
Transformers lack recurrence, so they rely on positional encodings to incorporate token order. You will implement **sinusoidal positional encoding**, as described in Vaswani et al. (2017), and add it to the input embeddings.

### Requirements
- Implement a function to generate sinusoidal positional encodings.
- Ensure the encoding is applied correctly to all input embeddings.

---

## Step 2: Implement Multi-Head Self-Attention
Instead of simply averaging word embeddings, you will replace this operation with **multi-head self-attention** to allow interactions between tokens.

### Requirements
- Implement a multi-head self-attention layer.
- Replace the `torch.mean(embedded, dim=1)` operation with the **attention mechanism**.
- Ensure that the implementation correctly processes padded tokens.

---


## Starter Code
Below is the base implementation of the model. You must complete the missing parts and train the model.

```python
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicEmbeddingModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes, num_heads):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=3)
        
        # Task 1: Implement positional encoding
        self.positional_encoding = PositionalEncoding(embed_dim)

        # Task 2: Implement Multi-Head Attention (replace mean pooling)
        # self.multihead_attn = ???

        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, input_ids, attention_mask, labels=None):
        embedded = self.embedding(input_ids)
        
        # Apply positional encoding
        embedded = self.positional_encoding(embedded)

        # Task: Replace mean pooling with Multi-Head Attention
        # attn_output, _ = ???(embedded, embedded, embedded, key_padding_mask=attention_mask)
        
        # Pooling: Extract the first token's representation (CLS token equivalent)
        # pooled_output = attn_output[:, 0, :]

        logits = self.fc(pooled_output)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}

# Students must implement this class
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=5000):
        super().__init__()
        self.encoding = self.create_positional_encoding(embed_dim, max_len)
        
    def create_positional_encoding(self, embed_dim, max_len):
        # Task: Implement sinusoidal positional encoding here
        pass

    def forward(self, x):
        # Task: Add positional encoding to embeddings
        pass
```

## Evaluation Criteria

To verify your implementation:

- Ensure that positional encoding correctly modifies embeddings.

- Confirm that the self-attention layer replaces mean pooling effectively.

- Train the modified model on a small sentiment dataset and compare results.

## Extra Challenges

For advanced students:

- Experiment with different pooling strategies (e.g., max pooling, CLS token extraction).

- Compare learned positional embeddings vs. sinusoidal encodings.

- Add layer normalization after attention.


## Step 1: Implement Positional Encoding

In [ ]:
# Model setup
embed_dim = 100
num_classes = 4
vocab_size = 1000

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Students must implement this class
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=500):
        super().__init__()
        self.encoding = self.create_positional_encoding(embed_dim, max_len)
        
    def create_positional_encoding(self, embed_dim, max_len):
        # Task: Implement sinusoidal positional encoding here
        position = np.arange(max_len)[:, np.newaxis]
        div_term = np.exp(np.arange(0, embed_dim, 2) * -(np.log(10000.0) / embed_dim))
        pe = np.zeros((max_len, embed_dim))
        pe[:, 0::2] = np.sin(position * div_term)
        pe[:, 1::2] = np.cos(position * div_term)
        pe = torch.from_numpy(pe).float()
        pe = pe.unsqueeze(0)  # Add batch dimension
        return pe

    def forward(self, x):
        self.encoding = self.encoding.to(x.device)
        # Task: Add positional encoding to embeddings
        x = x + self.encoding[:, :x.size(1)]
        return x

### Test Positional Encoding

In [55]:
def test_positional_encoding():
    # Parameters
    batch_size = 2
    seq_length = 5
    embed_dim = 10

    pos_enc = PositionalEncoding(embed_dim, max_len=seq_length)
    
    embeddings = torch.randn(batch_size, seq_length, embed_dim)
    
    encoded_embeddings = pos_enc(embeddings)

    # Validate dimensions
    assert embeddings.shape == encoded_embeddings.shape, "Dimensionality mismatch"

    # Check that same positional indices across sequences have the same encoding added
    test_embeddings = torch.ones(batch_size, seq_length, embed_dim)
    encoded_test_embeddings = pos_enc(test_embeddings)

    # Verify that positional encodings are similar for same positions
    pos_encoding_diff = (encoded_test_embeddings[0] - encoded_test_embeddings[1]).abs().max().item()
    assert pos_encoding_diff < 1e-5, f"Positional encodings vary across identical positions: {pos_encoding_diff}"
    
    print("All tests passed successfully.")

# Ejecutamos la prueba
test_positional_encoding()

All tests passed successfully.


## Step 2: Implement Multi-Head Self-Attention


In [ ]:
class BasicEmbeddingModelInproved(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes, num_heads):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=3)
        
        # Task 1: Implement positional encoding
        self.positional_encoding = PositionalEncoding(embed_dim)

        # Task 2: Implement Multi-Head Attention (replace mean pooling)
        self.multihead_attn = nn.MultiheadAttention(embed_dim, num_heads)

        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, input_ids, attention_mask, labels=None):
        embedded = self.embedding(input_ids)
        
        # Apply positional encoding
        embedded = self.positional_encoding(embedded)

        # Transpose needed for MultiheadAttention: (seq_len, batch_size, embed_dim)
        embedded = embedded.transpose(0, 1) #new code
        
        # attention_mask needs to be inverted for key_padding_mask
        key_padding_mask = ~attention_mask.bool() # new code

        # Task: Replace mean pooling with Multi-Head Attention
        attn_output, _ = self.multihead_attn(embedded, embedded, embedded, key_padding_mask=key_padding_mask)

        # Switching back to (batch_size, seq_len, embed_dim)
        attn_output = attn_output.transpose(0, 1) #new code

        # Pooling: Extract the first token's representation (CLS token equivalent)

        pooled_output = attn_output[:, 0, :]

        logits = self.fc(pooled_output)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}

## Train the modified model on a small sentiment dataset and compare results.

In [108]:
from transformers import EarlyStoppingCallback
def train_model(num_heads, ds_train, ds_test, fast_tokenizer):
    
    # Define Training Arguments
    training_args = TrainingArguments(
        output_dir="./results_{}".format(num_heads),
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=1e-3,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=50,
        load_best_model_at_end=True, 
    )
    # Model setup

    model_2 = BasicEmbeddingModelInproved(vocab_size, embed_dim, num_classes,num_heads=num_heads)
    # Show model summary
    print(model_2)

    # Define Trainer
    trainer = Trainer(
        model=model_2,
        args=training_args,
        train_dataset=ds_train,
        eval_dataset=ds_test,
        processing_class=fast_tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # Train model
    trainer.train()

    return trainer

In [110]:
for num_heads in [1, 2, 10,25,50,100,500]:
    print(f"Training model with {num_heads} heads")
    train_model(num_heads, ds_train, ds_test, fast_tokenizer)

Training model with 1 heads
BasicEmbeddingModelInproved(
  (embedding): Embedding(1000, 100, padding_idx=3)
  (positional_encoding): PositionalEncoding()
  (multihead_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
  )
  (fc): Linear(in_features=100, out_features=4, bias=True)
)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.488300,0.491868,0.825000,0.824607
2,0.431700,0.433494,0.847500,0.846552
3,0.412200,0.426875,0.852105,0.852274
4,0.395300,0.420548,0.851447,0.851536
5,0.395200,0.418065,0.850526,0.850885
6,0.382400,0.405483,0.851711,0.851218
7,0.395200,0.410620,0.855921,0.855792
8,0.372800,0.419779,0.852632,0.851679
9,0.388900,0.406382,0.855395,0.855294


Training model with 2 heads
BasicEmbeddingModelInproved(
  (embedding): Embedding(1000, 100, padding_idx=3)
  (positional_encoding): PositionalEncoding()
  (multihead_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
  )
  (fc): Linear(in_features=100, out_features=4, bias=True)
)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.483600,0.507013,0.823158,0.822999
2,0.433500,0.440952,0.843947,0.842958
3,0.414300,0.439832,0.848816,0.848987
4,0.398500,0.431093,0.847763,0.847863
5,0.397700,0.426572,0.850921,0.851020
6,0.384500,0.413845,0.854211,0.853682
7,0.397800,0.414575,0.853684,0.853629
8,0.374200,0.427230,0.849605,0.848686
9,0.391500,0.414728,0.854737,0.854449


Training model with 10 heads
BasicEmbeddingModelInproved(
  (embedding): Embedding(1000, 100, padding_idx=3)
  (positional_encoding): PositionalEncoding()
  (multihead_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
  )
  (fc): Linear(in_features=100, out_features=4, bias=True)
)


...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.455700,0.457944,0.831711,0.830976
2,0.416400,0.419770,0.846842,0.846238
3,0.395400,0.413358,0.849605,0.849860
4,0.375100,0.411707,0.851579,0.852138
5,0.361400,0.405729,0.857237,0.857253
6,0.344100,0.405391,0.853289,0.852385
7,0.346000,0.403650,0.854211,0.853848
8,0.325000,0.408529,0.852763,0.852122
9,0.322300,0.420796,0.852500,0.851982
10,0.297300,0.412230,0.855658,0.855308


Training model with 100 heads
BasicEmbeddingModelInproved(
  (embedding): Embedding(1000, 100, padding_idx=3)
  (positional_encoding): PositionalEncoding()
  (multihead_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
  )
  (fc): Linear(in_features=100, out_features=4, bias=True)
)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.449200,0.454630,0.837895,0.837819
2,0.408000,0.418975,0.847895,0.847473
3,0.387200,0.419678,0.851974,0.852338
4,0.363900,0.412922,0.851974,0.852375
5,0.352000,0.406112,0.863289,0.863149
6,0.325900,0.415946,0.855789,0.855329
7,0.329200,0.405048,0.858026,0.858276
8,0.301400,0.425316,0.853816,0.853594
9,0.302500,0.431360,0.858684,0.858658
10,0.271800,0.429966,0.853947,0.854021


Training model with 500 heads


AssertionError: embed_dim must be divisible by num_heads

### conclusión

Pese a agregar el enconder posicional y las capas de multihead attention, no se logro una mejora representativa en las metricas. Esto se puede deber a que se requieren mas datos de entrenamiento. Se adecuo una parada de epocas anticipada para evitar overfiting del modelo.

## Extra Challenges

* ### Experiment with different pooling strategies (e.g., max pooling, CLS token extraction).

In [111]:
class BasicEmbeddingModelInproved2(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes, num_heads, pooling_strategy="cls"):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=3)
        
        # Task 1: Implement positional encoding
        self.positional_encoding = PositionalEncoding(embed_dim)

        # Task 2: Implement Multi-Head Attention (replace mean pooling)
        self.multihead_attn = nn.MultiheadAttention(embed_dim, num_heads)

        self.fc = nn.Linear(embed_dim, num_classes)

         # Define la estrategia de pooling ("cls" o "max")
        if pooling_strategy not in ["cls", "max"]:
            raise ValueError("Pooling strategy must be either 'cls' or 'max'")
        self.pooling_strategy = pooling_strategy

    def forward(self, input_ids, attention_mask, labels=None):
        embedded = self.embedding(input_ids)
        
        # Apply positional encoding
        embedded = self.positional_encoding(embedded)

        # Transpose needed for MultiheadAttention: (seq_len, batch_size, embed_dim)
        embedded = embedded.transpose(0, 1) #new code
        
        # attention_mask needs to be inverted for key_padding_mask
        key_padding_mask = ~attention_mask.bool() # new code

        # Task: Replace mean pooling with Multi-Head Attention
        attn_output, _ = self.multihead_attn(embedded, embedded, embedded, key_padding_mask=key_padding_mask)

        # Switching back to (batch_size, seq_len, embed_dim)
        attn_output = attn_output.transpose(0, 1) #new code

        # Pooling: Extract the first token's representation (CLS token equivalent)

        # Estrategia de pooling
        if self.pooling_strategy == "cls":
            # Extraer la representación del primer token
            pooled_output = attn_output[:, 0, :]
        elif self.pooling_strategy == "max":
            # Max Pooling a través de la dimensión de secuencia
            pooled_output = torch.max(attn_output, dim=1)[0]

        logits = self.fc(pooled_output)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}

In [115]:
from transformers import EarlyStoppingCallback
def train_model(pooling_strategy, ds_train, ds_test, fast_tokenizer):
    
    # Define Training Arguments
    training_args = TrainingArguments(
        output_dir="./results_{}".format(num_heads),
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=1e-3,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=50,
        load_best_model_at_end=True, 
    )
    # Model setup

    model3 = BasicEmbeddingModelInproved2(vocab_size, embed_dim, num_classes, num_heads=2, pooling_strategy=pooling_strategy)
    # Show model summary
    print(model3)

    # Define Trainer
    trainer = Trainer(
        model=model3,
        args=training_args,
        train_dataset=ds_train,
        eval_dataset=ds_test,
        processing_class=fast_tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # Train model
    trainer.train()

    return trainer

In [117]:
for pooling_strategy in ["cls", "max"]:
    print(f"Training model with pooling_strategy {pooling_strategy}")
    train_model(pooling_strategy, ds_train, ds_test, fast_tokenizer)

Training model with pooling_strategy cls
BasicEmbeddingModelInproved2(
  (embedding): Embedding(1000, 100, padding_idx=3)
  (positional_encoding): PositionalEncoding()
  (multihead_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
  )
  (fc): Linear(in_features=100, out_features=4, bias=True)
)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.484800,0.479975,0.830000,0.829435
2,0.431900,0.430264,0.844737,0.843600
3,0.411700,0.432386,0.848289,0.848414
4,0.396600,0.421190,0.851316,0.851448
5,0.395800,0.418295,0.854211,0.854139
6,0.381500,0.404786,0.856974,0.856601
7,0.393000,0.410053,0.857368,0.857286
8,0.371900,0.421451,0.851579,0.850849
9,0.387100,0.405998,0.855000,0.854819


Training model with pooling_strategy max
BasicEmbeddingModelInproved2(
  (embedding): Embedding(1000, 100, padding_idx=3)
  (positional_encoding): PositionalEncoding()
  (multihead_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
  )
  (fc): Linear(in_features=100, out_features=4, bias=True)
)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.482200,0.486686,0.821053,0.821056
2,0.430100,0.449983,0.833684,0.832066
3,0.398300,0.417241,0.851842,0.852279
4,0.367600,0.411708,0.852763,0.852626
5,0.352900,0.400288,0.858553,0.858537
6,0.336300,0.396893,0.859868,0.859686
7,0.333700,0.391645,0.861842,0.861770
8,0.309000,0.423764,0.852237,0.851271
9,0.308000,0.407651,0.861842,0.861832
10,0.277500,0.414494,0.859737,0.859191


* ### Compare learned positional embeddings vs. sinusoidal encodings.

In [133]:
import torch
import torch.nn as nn
import math

class PositionalEncodingInproved(nn.Module):
    def __init__(self, embed_dim, max_len=5000, method="sinusoidal"):
        super().__init__()
        self.method = method
        assert method in ["sinusoidal", "learned"], "Method should be 'sinusoidal' or 'learned'"
        
        if method == "sinusoidal":
            # Create a constant positional encoding matrix with sinusoidal values
            position = torch.arange(0, max_len).unsqueeze(1).float()
            div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * -(math.log(10000.0) / embed_dim))
            pe = torch.zeros(max_len, 1, embed_dim)
            pe[:, 0, 0::2] = torch.sin(position * div_term)
            pe[:, 0, 1::2] = torch.cos(position * div_term)
            self.register_buffer('pe', pe)
        elif method == "learned":
            # Use nn.Embedding to learn positional encodings
            self.position_embedding = nn.Embedding(max_len, embed_dim)
        
    def forward(self, x):
        if self.method == "sinusoidal":
            x = x + self.pe[:x.size(1)]
        elif self.method == "learned":
            positions = torch.arange(x.size(1), device=x.device).unsqueeze(0).expand(x.size(0), -1)
            x = x + self.position_embedding(positions)
        return x

class BasicEmbeddingModelInproved3(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes, num_heads, max_len=5000, pooling_strategy="cls", pe_method="sinusoidal"):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=3)
        self.positional_encoding = PositionalEncodingInproved(embed_dim, max_len, method=pe_method)
        self.multihead_attn = nn.MultiheadAttention(embed_dim, num_heads)
        self.fc = nn.Linear(embed_dim, num_classes)
        
        assert pooling_strategy in ["cls", "max"], "pooling_strategy debe ser 'cls' o 'max'"
        self.pooling_strategy = pooling_strategy

    def forward(self, input_ids, attention_mask, labels=None):
        embedded = self.embedding(input_ids)
        embedded = self.positional_encoding(embedded)
        embedded = embedded.transpose(0, 1)

        key_padding_mask = ~attention_mask.bool()
        attn_output, _ = self.multihead_attn(embedded, embedded, embedded, key_padding_mask=key_padding_mask)
        attn_output = attn_output.transpose(0, 1)

        if self.pooling_strategy == "cls":
            pooled_output = attn_output[:, 0, :]
        elif self.pooling_strategy == "max":
            pooled_output = torch.max(attn_output, dim=1)[0]

        logits = self.fc(pooled_output)
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}


In [146]:
from transformers import EarlyStoppingCallback
def train_model(ds_train, ds_test,fast_tokenizer, max_len=5000, pooling_strategy="max", pe_method="learned",num_heads=2):
    
    # Define Training Arguments
    training_args = TrainingArguments(
        output_dir="./results_{}".format(num_heads),
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=1e-3,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=50,
        load_best_model_at_end=True, 
    )
    # Model setup

   
    model4 = BasicEmbeddingModelInproved3(vocab_size, embed_dim, num_classes, max_len = max_len, num_heads=num_heads, 
                                    pooling_strategy=pooling_strategy, pe_method=pe_method)
    # Show model summary
    print(model4)

    # Define Trainer
    trainer = Trainer(
        model=model4,
        args=training_args,
        train_dataset=ds_train,
        eval_dataset=ds_test,
        processing_class=fast_tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # Train model
    trainer.train()

    return trainer

In [147]:
train_model(ds_train, ds_test, fast_tokenizer,max_len=5,pooling_strategy="max", pe_method="learned",num_heads=2)

BasicEmbeddingModelInproved3(
  (embedding): Embedding(1000, 100, padding_idx=3)
  (positional_encoding): PositionalEncodingInproved(
    (position_embedding): Embedding(5, 100)
  )
  (multihead_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
  )
  (fc): Linear(in_features=100, out_features=4, bias=True)
)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.449400,0.454042,0.839211,0.838656
2,0.409600,0.419236,0.850658,0.850111
3,0.382400,0.408171,0.856316,0.856793
4,0.358700,0.403927,0.857105,0.857138
5,0.349300,0.404277,0.861974,0.861823
6,0.329500,0.389227,0.861579,0.861091
7,0.327300,0.395465,0.864605,0.864582
8,0.300900,0.415779,0.857105,0.856392
9,0.303000,0.413410,0.863553,0.863179


In [148]:
train_model(ds_train, ds_test, fast_tokenizer,max_len=5,pooling_strategy="max", pe_method="learned",num_heads=10)

BasicEmbeddingModelInproved3(
  (embedding): Embedding(1000, 100, padding_idx=3)
  (positional_encoding): PositionalEncodingInproved(
    (position_embedding): Embedding(5, 100)
  )
  (multihead_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
  )
  (fc): Linear(in_features=100, out_features=4, bias=True)
)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.439100,0.437550,0.842632,0.841923
2,0.391100,0.403842,0.853816,0.853051
3,0.367700,0.392838,0.856842,0.856735
4,0.336200,0.390869,0.865000,0.865433
5,0.318700,0.405102,0.862237,0.862278
6,0.294500,0.395169,0.861184,0.860743
7,0.283900,0.416794,0.858421,0.858639


In [149]:
train_model(ds_train, ds_test, fast_tokenizer,max_len=10,pooling_strategy="max", pe_method="learned",num_heads=2)

BasicEmbeddingModelInproved3(
  (embedding): Embedding(1000, 100, padding_idx=3)
  (positional_encoding): PositionalEncodingInproved(
    (position_embedding): Embedding(10, 100)
  )
  (multihead_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
  )
  (fc): Linear(in_features=100, out_features=4, bias=True)
)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.466000,0.468439,0.830263,0.829341
2,0.412500,0.427626,0.847763,0.847307
3,0.388200,0.411525,0.851974,0.851678
4,0.358600,0.410420,0.853553,0.854024
5,0.350100,0.408897,0.859342,0.859048
6,0.326900,0.402689,0.859079,0.858843
7,0.337100,0.404840,0.861711,0.861642
8,0.305000,0.423911,0.855921,0.855372
9,0.304300,0.419663,0.861053,0.860986


* ### Add layer normalization after attention.

In [150]:
import torch
import torch.nn as nn
import math

class PositionalEncodingInproved(nn.Module):
    def __init__(self, embed_dim, max_len=500, method="learned"):
        super().__init__()
        self.method = method
        assert method in ["sinusoidal", "learned"], "Method should be 'sinusoidal' or 'learned'"
        
        if method == "sinusoidal":
            # Create a constant positional encoding matrix with sinusoidal values
            position = torch.arange(0, max_len).unsqueeze(1).float()
            div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * -(math.log(10000.0) / embed_dim))
            pe = torch.zeros(max_len, 1, embed_dim)
            pe[:, 0, 0::2] = torch.sin(position * div_term)
            pe[:, 0, 1::2] = torch.cos(position * div_term)
            self.register_buffer('pe', pe)
        elif method == "learned":
            # Use nn.Embedding to learn positional encodings
            self.position_embedding = nn.Embedding(max_len, embed_dim)
        
    def forward(self, x):
        if self.method == "sinusoidal":
            x = x + self.pe[:x.size(1)]
        elif self.method == "learned":
            positions = torch.arange(x.size(1), device=x.device).unsqueeze(0).expand(x.size(0), -1)
            x = x + self.position_embedding(positions)
        return x

class BasicEmbeddingModelInproved4(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes, num_heads, max_len=5000, pooling_strategy="cls", pe_method="learned"):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=3)
        self.positional_encoding = PositionalEncodingInproved(embed_dim, max_len, method=pe_method)
        self.multihead_attn = nn.MultiheadAttention(embed_dim, num_heads)
        # Add LayerNorm after attention
        self.layer_norm = nn.LayerNorm(embed_dim)
        self.fc = nn.Linear(embed_dim, num_classes)
        
        assert pooling_strategy in ["cls", "max"], "pooling_strategy debe ser 'cls' o 'max'"
        self.pooling_strategy = pooling_strategy

    def forward(self, input_ids, attention_mask, labels=None):
        embedded = self.embedding(input_ids)
        embedded = self.positional_encoding(embedded)
        embedded = embedded.transpose(0, 1)

        key_padding_mask = ~attention_mask.bool()
        attn_output, _ = self.multihead_attn(embedded, embedded, embedded, key_padding_mask=key_padding_mask)
        attn_output = attn_output.transpose(0, 1)

        if self.pooling_strategy == "cls":
            pooled_output = attn_output[:, 0, :]
        elif self.pooling_strategy == "max":
            pooled_output = torch.max(attn_output, dim=1)[0]

        logits = self.fc(pooled_output)
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}

In [151]:
from transformers import EarlyStoppingCallback
def train_model(ds_train, ds_test,fast_tokenizer, max_len=50, pooling_strategy="max", pe_method="learned",num_heads=2):
    
    # Define Training Arguments
    training_args = TrainingArguments(
        output_dir="./results_{}".format(num_heads),
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=1e-3,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=50,
        load_best_model_at_end=True, 
    )
    # Model setup

   
    model4 = BasicEmbeddingModelInproved4(vocab_size, embed_dim, num_classes, max_len = max_len, num_heads=num_heads, 
                                    pooling_strategy=pooling_strategy, pe_method=pe_method)
    # Show model summary
    print(model4)

    # Define Trainer
    trainer = Trainer(
        model=model4,
        args=training_args,
        train_dataset=ds_train,
        eval_dataset=ds_test,
        processing_class=fast_tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # Train model
    trainer.train()

    return trainer

In [152]:
train_model(ds_train, ds_test, fast_tokenizer,max_len=50,pooling_strategy="max", pe_method="learned",num_heads=2)

BasicEmbeddingModelInproved4(
  (embedding): Embedding(1000, 100, padding_idx=3)
  (positional_encoding): PositionalEncodingInproved(
    (position_embedding): Embedding(50, 100)
  )
  (multihead_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
  )
  (layer_norm): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
  (fc): Linear(in_features=100, out_features=4, bias=True)
)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.481400,0.469952,0.828421,0.827804
2,0.429100,0.432803,0.843553,0.843225
3,0.403000,0.422411,0.847500,0.847760
4,0.378800,0.411985,0.854737,0.854619
5,0.367400,0.405253,0.855132,0.855046
6,0.353900,0.403804,0.858158,0.857430
7,0.357600,0.396494,0.859211,0.858997
8,0.335300,0.443442,0.851579,0.850720
9,0.335100,0.404788,0.863816,0.863706
10,0.304800,0.414321,0.859868,0.859464


### conclusion: 

Durante las pruebas, se experimentó con varias configuraciones del modelo para mejorar la precisión y el puntaje F1. Se implementaron diferentes estrategias de pooling, incluyendo max pooling y extracción del token CLS, pero no se observaron mejoras significativas en el rendimiento. Además, se añadió normalización por capas (Layer Normalization) después de la capa de atención, y se compararon distintos métodos de codificación posicional, como learned positional embeddings VS codificaciones sinusoidales.

En los resultados no se observa un aumento notable en la precisión ni en el puntaje F1. Esto sugiere que podría ser necesario realizar un ajuste más fino de los hiperparámetros. Variables como la tasa de aprendizaje, el número de cabezas de atención (num_heads), y el tamaño de los embeddings (embed_dim) deben ser exploradas con técnicas grid search o random search para encontrar configuraciones más óptimas.

Otra posibilidad es que los datos de entrenamiento actuales no sean suficientes para que el modelo aprenda de manera efectiva.

In [171]:
import random
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'max_len': [32, 64, 96, 128],
    'learning_rate': [1e-4, 5e-3, 1e-2],
    'pooling_strategy': ["max", "cls"],
    'pe_method': ["learned", "sinusoidal"],
    'batch_size': [8, 16, 32, 64]
}

# Ajuste inicial para embed_dim y num_heads
possible_heads = [2, 4, 8]  
embed_dim_choices = [64, 128, 256, 512]  

# Combinaciones válidas
valid_combinations = [(embed_dim, num_heads) for embed_dim in embed_dim_choices for num_heads in possible_heads if embed_dim % num_heads == 0]

# Definir la métrica de mejor resultado
best_accuracy = 0
best_params = {}

# Conjunto para almacenar combinaciones ya probadas
tested_combinations = set()

# Realizar Random Search con un número determinado de intentos
n_trials = 20

while len(tested_combinations) < n_trials:
    # Seleccionar aleatoriamente los hiperparámetros
    max_len = random.choice(param_grid['max_len'])
    learning_rate = random.choice(param_grid['learning_rate'])
    pooling_strategy = random.choice(param_grid['pooling_strategy'])
    pe_method = random.choice(param_grid['pe_method'])
    batch_size = random.choice(param_grid['batch_size'])

    # Selecciona una combinación válida de embed_dim y num_heads
    embed_dim, num_heads = random.choice(valid_combinations)

    # Crear una tupla de la combinación actual
    current_combination = (
        max_len, 
        learning_rate, 
        pooling_strategy, 
        pe_method, 
        batch_size, 
        embed_dim, 
        num_heads
    )
    
    # Verificar si la combinación ya ha sido probada
    if current_combination in tested_combinations:
        continue
    
    # Añadir la combinación actual al conjunto de combinaciones probadas
    tested_combinations.add(current_combination)

    print({
        'max_len': max_len,
        'learning_rate': learning_rate,
        'num_heads': num_heads,
        'pooling_strategy': pooling_strategy,
        'pe_method': pe_method,
        'batch_size': batch_size
    })
    
    # Configuración del entrenamiento
    training_args = TrainingArguments(
        output_dir=f"./results_{num_heads}",
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=50,
        load_best_model_at_end=True, 
    )
    
    # Configuración del modelo
    model4 = BasicEmbeddingModelInproved4(
        vocab_size=vocab_size, 
        embed_dim=embed_dim, 
        num_classes=num_classes,
        max_len=max_len, 
        num_heads=num_heads, 
        pooling_strategy=pooling_strategy, 
        pe_method=pe_method
    )
    
    # Definir Trainer
    trainer = Trainer(
        model=model4,
        args=training_args,
        train_dataset=ds_train,
        eval_dataset=ds_test,
        processing_class=fast_tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    
    try:
        # Entrenar y evaluar el modelo
        trainer.train()
        eval_results = trainer.evaluate()
        
        # Comparar y guardar los mejores parámetros
        if eval_results["eval_accuracy"] > best_accuracy:
            best_accuracy = eval_results["eval_accuracy"]
            best_params = {
                'max_len': max_len,
                'learning_rate': learning_rate,
                'num_heads': num_heads,
                'pooling_strategy': pooling_strategy,
                'pe_method': pe_method,
                'batch_size': batch_size
            }
    except Exception as e:
        print(f"Error en la combinación {current_combination}: {e}")
        continue

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", best_params)
print("Mejor precisión:", best_accuracy)

{'max_len': 32, 'learning_rate': 0.0001, 'num_heads': 8, 'pooling_strategy': 'cls', 'pe_method': 'learned', 'batch_size': 16}


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.656800,0.647464,0.758947,0.757945
2,0.542500,0.544942,0.805000,0.804037
3,0.489400,0.504930,0.820132,0.819745
4,0.473900,0.490307,0.827632,0.827520
5,0.425700,0.478359,0.829211,0.828759
6,0.439200,0.464823,0.834342,0.833787
7,0.407800,0.461696,0.838421,0.838432
8,0.411200,0.463692,0.837895,0.836944
9,0.390100,0.449961,0.846053,0.845673
10,0.396600,0.444510,0.844342,0.844242


{'max_len': 32, 'learning_rate': 0.01, 'num_heads': 2, 'pooling_strategy': 'max', 'pe_method': 'sinusoidal', 'batch_size': 8}
Error en la combinación (32, 0.01, 'max', 'sinusoidal', 8, 64, 2): The size of tensor a (8) must match the size of tensor b (32) at non-singleton dimension 0
{'max_len': 32, 'learning_rate': 0.0001, 'num_heads': 8, 'pooling_strategy': 'max', 'pe_method': 'learned', 'batch_size': 16}


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.444700,0.438722,0.840658,0.840125
2,0.374200,0.398508,0.858553,0.858050
3,0.325000,0.397276,0.864474,0.864720
4,0.278600,0.435335,0.862500,0.862825
5,0.211400,0.476803,0.862895,0.862645
6,0.177800,0.543318,0.860526,0.860192


{'max_len': 128, 'learning_rate': 0.0001, 'num_heads': 8, 'pooling_strategy': 'cls', 'pe_method': 'sinusoidal', 'batch_size': 8}
Error en la combinación (128, 0.0001, 'cls', 'sinusoidal', 8, 64, 8): The size of tensor a (8) must match the size of tensor b (128) at non-singleton dimension 0
{'max_len': 128, 'learning_rate': 0.005, 'num_heads': 8, 'pooling_strategy': 'cls', 'pe_method': 'learned', 'batch_size': 16}


...

{'max_len': 32, 'learning_rate': 0.005, 'num_heads': 8, 'pooling_strategy': 'cls', 'pe_method': 'sinusoidal', 'batch_size': 32}


Epoch,Training Loss,Validation Loss


Error en la combinación (32, 0.005, 'cls', 'sinusoidal', 32, 256, 8): The size of tensor a (16) must match the size of tensor b (32) at non-singleton dimension 0
{'max_len': 32, 'learning_rate': 0.01, 'num_heads': 4, 'pooling_strategy': 'max', 'pe_method': 'sinusoidal', 'batch_size': 32}


Epoch,Training Loss,Validation Loss


Error en la combinación (32, 0.01, 'max', 'sinusoidal', 32, 64, 4): The size of tensor a (16) must match the size of tensor b (32) at non-singleton dimension 0
Mejores hiperparámetros: {'max_len': 32, 'learning_rate': 0.0001, 'num_heads': 8, 'pooling_strategy': 'max', 'pe_method': 'learned', 'batch_size': 16}
Mejor precisión: 0.8644736842105263


In [174]:
mejora_porcentual = ((0.8644736842105263-0.852500)/0.852500)*100
print(mejora_porcentual)

1.4045377373051318


### conclusion ajuste hiperparametros
Con un ajuste de hiperparametros , se optiene una mejora en la precision de de 1.4% , lo cual no es relevante versus el aumento de complejidad del modelo. Y confirma que se deben aumentar la cantidad de datos para poder alcancer una metrica mayor.
Mejores hiperparámetros: {'max_len': 32, 'learning_rate': 0.0001, 'num_heads': 8, 'pooling_strategy': 'max', 'pe_method': 'learned', 'batch_size': 16}
Mejor precisión: 0.8644736842105263